# Make Bing API Calls and save the API results in a lakehouse table
This code demonstrates how to use notebooks to make bing api calls
## Prerequisites
To run the code , add a lakehouse from the lakehouse explorer

In [ ]:
import json
import os
import requests

#make REST Api call to bing search 
search_url='https://api.bing.microsoft.com/v7.0/search'
subscription_key=''#your bing key here
mkt = 'en-US'
query='current weather in LA'#query to search via bing
params = { 'q': query, 'mkt': mkt ,"count":50, "answerCount":10 ,"textDecorations": True, "textFormat": "HTML","responseFilter":"webpages" }#bing restapi params
headers = { 'Ocp-Apim-Subscription-Key': subscription_key }#bing auth header


In [ ]:
#curate the response fron bing search to dataframe
#api calls to bing and create dataframe from the response
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
data= (response.json())
df_bing = spark.createDataFrame(data['webPages']['value']).select('name','snippet','url')
display(df_bing)

In [ ]:
#enable vordering for performance and set delta parquet file size
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

In [ ]:
#write dataframe to table 
df_bing.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/bingresults")